# Connect to ElasticSearch Serverless project

This notebook aims to provide a few examples on how to interact with an ElasticSearch Serverless project.

## Import packages

In [1]:
import os
from datetime import datetime

import requests
from elasticsearch_serverless import Elasticsearch

# add here any package you need to use

## Connection Details

We have created manually a serverless project. This process can also be automated by using the [Project Management REST API](https://www.elastic.co/guide/en/serverless/current/general-manage-project-with-api.html).

In [ ]:
ELASTICSEARCH_ENDPOINT = "https://ml-rd-ops-e3b5cc.es.us-east-1.aws.elastic.cloud:443"
API_KEY = os.environ[
    "ES_SERVERLESS_API_KEY"
]  # you will need to set this environment variable with your API Key.

## ElasticSearch Serverless client

We are going to use the [ElasticSearch Python client](https://www.elastic.co/guide/en/serverless/current/elasticsearch-python-client-getting-started.html) whenever possible. 

In [4]:
client = Elasticsearch(ELASTICSEARCH_ENDPOINT, api_key=API_KEY)
client.info()

ObjectApiResponse({'name': 'serverless', 'cluster_name': 'b876c513c5134470bee831f9762f9f2c', 'cluster_uuid': 'u5yIcSZ3Q6WRX8LdQ83MWw', 'version': {'number': '8.11.0', 'build_flavor': 'serverless', 'build_type': 'docker', 'build_hash': '00000000', 'build_date': '2023-10-31', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '8.11.0', 'minimum_index_compatibility_version': '8.11.0'}, 'tagline': 'You Know, for Search'})

## Index creation and bulk ingestion

Let's index some sample data using the bulk API:

In [5]:
client.bulk(
    body=[
        {"index": {"_index": "books", "_id": "1"}},
        {
            "title": "Infinite Jest",
            "author": "David Foster Wallace",
            "published_on": datetime(1996, 2, 1),
        },
        {"index": {"_index": "books", "_id": "2"}},
        {"title": "Ulysses", "author": "James Joyce", "published_on": datetime(1922, 2, 2)},
        {"index": {"_index": "books", "_id": "3"}},
        {"title": "Just Kids", "author": "Patti Smith", "published_on": datetime(2010, 1, 19)},
    ],
)

ObjectApiResponse({'errors': False, 'took': 1600, 'items': [{'index': {'_index': 'books', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'books', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'books', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}]})

## Search data and score using BM25

Now we can search data using the Search API and get the relevance score for each result. The relevance score is calculated by default using the BM25 algorithm.

In [6]:
response = client.search(index="books", query={"match": {"title": "infinite"}})

for hit in response["hits"]["hits"]:
    print(hit)

## Search data and score using ELSERv2

Instead of using BM25, we can retrieve the results using ELSERv2.

In order to so that, we need to prepare the index mappings first:

In [7]:
es_url = f"{ELASTICSEARCH_ENDPOINT}/books_sparse"

headers = {"Authorization": f"ApiKey {API_KEY}", "Content-Type": "application/json"}

data = {
    "mappings": {
        "properties": {"sparse_embeddings": {"type": "sparse_vector"}, "title": {"type": "text"}}
    }
}

response = requests.put(es_url, headers=headers, json=data)

if response.status_code == 200:
    print(f"Index created successfully: {response.json()}")
else:
    print(f"Error when creating the index: {response.status_code}")
    print(f"Error details: {response.text}")

Index created successfully: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'books_sparse'}


Now let's generate the text embeddings using an ingest pipeline with an inference processor. The inference processor will use ELSERv2 inference to compute the text embeddings.

This step requires ELSERv2 to be downloaded and deployed. We have done it manually in the UI, though this process can also be automated. The model deployment is optimized for ingesting purposes. Additionally, we will have another deployment optimized for search.

Let's create the ingest pipeline:

In [8]:
es_url = f"{ELASTICSEARCH_ENDPOINT}/_ingest/pipeline/generate-elserv2-embeddings-pipeline"

headers = {"Authorization": f"ApiKey {API_KEY}", "Content-Type": "application/json"}

data = {
    "description": "Generates text embeddings using ELSERv2",
    "processors": [
        {
            "inference": {
                "model_id": ".elser_model_2_linux-x86_64_ingest",
                "input_output": [{"input_field": "title", "output_field": "sparse_embeddings"}],
            }
        }
    ],
}

response = requests.put(es_url, headers=headers, json=data)

if response.status_code == 200:
    print(f"Ingest pipeline successfully created: {response.json()}")
else:
    print(f"Error when creating the ingesting pipeline: {response.status_code}")
    print(f"Error details: {response.text}")

Ingest pipeline successfully created: {'acknowledged': True}


Now let's reindex our books *index* to compute the embeddings for the *title* field.

In [10]:
es_url = f"{ELASTICSEARCH_ENDPOINT}/_reindex?wait_for_completion=true"

headers = {"Authorization": f"ApiKey {API_KEY}", "Content-Type": "application/json"}


data = {
    "source": {
        "index": "books",
        "size": 50,  # batch size for reindexing, the smaller, the quicker
    },
    "dest": {"index": "books_sparse", "pipeline": "generate-elserv2-embeddings-pipeline"},
}

response = requests.post(es_url, headers=headers, json=data)

if response.status_code == 200:
    print(f"Reindex completed successfully: {response.json()}")
else:
    print(f"Failed to complete reindexing operation. Status code: {response.status_code}")
    print(f"Details: {response.text}")

Reindex completed successfully: {'took': 706, 'timed_out': False, 'total': 3, 'updated': 0, 'created': 3, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


Once this data is ingested, we can perform semantic search with ELSERv2 as follows:

In [16]:
es_url = f"{ELASTICSEARCH_ENDPOINT}/books_sparse/_search"


headers = {"Authorization": f"ApiKey {API_KEY}", "Content-Type": "application/json"}

data = {
    "query": {
        "sparse_vector": {
            "field": "sparse_embeddings",
            "inference_id": ".elser_model_2_linux-x86_64_search",
            "query": "boundless",
        }
    }
}

response = requests.get(es_url, headers=headers, json=data)

if response.status_code == 200:
    print(f"Search query executed successfully: {response.json()}")
    print(
        f"Most relevant retrieved `title`: {response.json()['hits']['hits'][0]['_source']['title']}"
    )
else:
    print(f"Failed to execute search query. Status code: {response.status_code}")
    print(f"Details: {response.text}")

Search query executed successfully: {'took': 35, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 2.5879898, 'hits': [{'_index': 'books_sparse', '_id': '1', '_score': 2.5879898, '_source': {'published_on': '1996-02-01T00:00:00', 'sparse_embeddings': {'voice': 0.06321447, 'magic': 0.040825274, 'utter': 0.9487267, 'crowd': 0.015823865, 'pun': 0.2428866, 'ego': 0.0419161, 'anger': 0.68023884, 'improvisation': 0.15309939, 'character': 0.28845063, 'humor': 1.0575207, 'crush': 0.16111338, 'jam': 0.35862505, 'lust': 0.12904637, 'silence': 0.3605276, 'because': 0.120862685, 'fear': 0.12770632, 'song': 0.29209006, 'always': 0.18514793, 'audience': 0.03697875, 'chaos': 0.26750943, 'maxim': 0.3806816, 'ritual': 0.11960416, 'comedy': 0.23235129, 'parody': 0.40718827, '(': 0.12757999, 'protest': 0.15314254, 'animation': 0.26009154, 'rude': 0.49948692, 'distraction': 0.26121494, 'je': 2.5981421, 'm